# River load sensor calibration

![](./images/station-hydrologique-digue.jpeg) 
Picture of an hydrological station located near La Digue on the Capesterre river © ObsEra.

The original version of this notebook was made by Antoine Lucas on top of the study made by Amande Roque-Bernard with the help of Gregory Sainton. In order to know more about the scientific context, please refer to Roque-Bernard et al. ([2023](https://doi.org/10.5194/esurf-11-363-2023)) _Phenomenological model of suspended sediment transport in a small catchment_, Earth Surface Dynamics, 11, 363–381. The dataset comes from [ObsEra](https://www.ozcar-ri.org/fr/observatoire-obsera/) (OBServatoire de l’Eau et de l’éRosion aux Antilles), an observatory located in Guadeloupe that gives us information on the erosion of this volcanic island in a few watersheds. With this notebook, you will learn how to prepare and clean a dataset.

The current notebook was edited in 2023 by Léonard Seydoux (seydoux@ipgp.fr) for the course _Earth Data Science_ at the [Institut de Physique du Globe de Paris](https://www.ipgp.fr/fr) (IPGP). If you have found a bug or have a suggestion, please feel free to contact me.

<img src="images/logo-obsera.png" style="margin:20px; height:100px;"/> <img src="images/logo-ipgp-upc.png" height=100 style="margin:20px; height:100px;"/>

## Instructions

This notebook contains a series of questions to which you should answer. Some example cells are given in the notebook to help you. You should not modify these cells. You can add as many cells as you want to answer the questions. You can also add markdown cells to explain your approach. 

## Prerequisites

Before running the notebook, you should make sure that the necessary packages are installed. You can do this by running the following cell. Be sure you are using a correct environment (check the top right corner of the notebook for the name of the environment). Note that there is a blank line between the first import and the other ones. This is because we follow the [PEP8](https://www.python.org/dev/peps/pep-0008/) convention between standard library imports and third party imports.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model

## 1. Introduction

Estimating the suspended load within a river is a key issue in geomorphology. The suspended load is often estimated by sampling the water column and measuring the concentration $C_S$ of suspended particles. Although this method is accurate, it is not suitable for long-term monitoring, since it is time-consuming and expensive. A cheaper and faster method is to use turbidity probes. 

Turbidity probes allow to measure the turbidity $T$ of the water column, which is related to the concentration of suspended particles. However, the relationship between turbidity and suspended load is not straightforward. It depends on the size and the shape of the particles. The goal of this notebook is to calibrate a turbidity probe placed in a river against independent measurements of suspended load. In other terms, we want to find a relationship between the turbidity and the suspended load. And for this, we will use a machine-learning approach.

> __Question 1.__ What kind of machine learning problem is this? What is the input $x$, the output $y$? What are the features, the labels, what are the dimensions of the features and the labels spaces? 

The dataset comes from the _OBServatoire de l’Eau et de l’éRosion aux Antilles_ ([ObsERA](https://www.ozcar-ri.org/fr/observatoire-obsera/)), an observatory located in Guadeloupe that gives us information on the erosion of this volcanic island in a few watersheds. With this notebook, you will learn how to prepare and clean a dataset. It is likely the most important part of the job. Then, you will play with your first models with the `scikit-learn` library. 

This notebook is mostly dedicated to the dataset preparation and cleaning. This is likely the most important part of the job. Then, you will play with your first models with the `scikit-learn` library. 

## 2. Suspended load measurements

The data for the calibration are stored under the [`./data`](./data) directory. The data are separated in two subsets, one with chemical information which contains the independent measurements of suspended load, and another one with hydrological information which contains the turbidity and water level measurements. 

### 2.1. Load the chemical data

The chemical data comes in a single CSV file located under the [`./data/chemistry`](./data/chemistry) repository. We here load it in a Pandas dataframe with the `pd.read_csv()` function. The goal of this first cell is to correctly parse the data, especially by converting the date and time information into `pd.Timestamp` objects and by selecting the column of interest. Note that the last statement of a Jupyter cell is automatically printed in a better format. 

> __Question 2.__ What is the `delimiter` keyword argument made for, and why did we set it to a semicolon `;` to load the data? How many rows do we have in this first dataset? And columns? 

In [ ]:
# Read it into a pandas dataframe
data_chemical_original = pd.read_csv(
    "./data/chemistry/CE_DIG_OBSERA_RIVERS_2019-09-26.csv",
    delimiter=";",
)

# Print the first 5 rows
data_chemical_original.head()

### 2.2. Turn the index into a datetime index

Note that pandas DataFrames have a special column called the index. You can see above that this column does not have any name and is used to uniquely identify each row of the DataFrame. By default, the index is a sequence of integers from $0$ to $N-1$, where $N$ is the number of rows in the DataFrame. You can also use the date and time information as the index of the DataFrame. This is useful when you want to select a subset of the data based on date and time information. The following cell shows how to do this. Check the new index column.

In [ ]:
# Copy for safety
data_chemical_copy = data_chemical_original.copy()

# Merge the date and time columns together
date_and_time = data_chemical_copy["Date"] + " " + data_chemical_copy["Hour"]

# Convert the merge into a datetime column
data_chemical_copy["datetime"] = pd.to_datetime(date_and_time, format="mixed")

# Set the datetime column as the index
# The inplace=True keyword argument allows to replace the existing dataframe
data_chemical_copy.set_index("datetime", inplace=True)
data_chemical_copy.sort_index(inplace=True)

# Drop the old date and time columns from the oringal dataframe
data_chemical_copy.drop(["Date", "Hour"], axis=1, inplace=True)

# Check head
data_chemical_copy.head()

### 2.3. Select the suspended load column

Finally, we extract the data of interest here, a.k.a. the suspended load measurements. Note that since this is only a single column from the `data_chemical_copy` DataFrame, the result is a pandas Series.

In [ ]:
# Extract the suspended load
suspended_load = data_chemical_copy["Suspended Load"]

# Drop the NaN values
suspended_load.dropna(inplace=True)

# Print the first few rows
suspended_load.head()

Pandas is a very handy library to inspect and manipulate data. You can find more information about it in the [documentation](https://pandas.pydata.org/pandas-docs/stable/). For instance, some high-level methods allow you to quickly inspect the data. For instance, the `describe()` method gives you a summary of the data. The `plot()` method allows you to quickly plot the data. You can also use the `head()` and `tail()` methods to inspect the first and last rows of the DataFrame. 

> __Question 3.__ Can you give a high-level definition of what an object is in Python? This definition should include the notion of _attributes_ and _methods_. According to this definition, what is a DataFrame, and what does the `info()` instruction do?

In [ ]:
# Plot
suspended_load.plot(
    style=".",
    xlabel="Date",
    ylabel="$C_S$ (mg/L)",
    grid=True,
    title="Suspended load as a function of time",
)

# Describe the data
suspended_load.describe()

## 3. Hydrological data

Your turn now! In this part, we will deal with the hydrological data. The ultimate goal of this practical is to use some of the hydrological data to predict the suspended load data. Thus, we need to load the hydrological data into a dataframe. Note that the hydrological data are split into different files. We thus first need to load all the files individually with the `pd.read_csv()` function, and then concatenate them into a single DataFrame with the `pd.concat()` function.

First, in order to list all hydrological, you can make use of the `glob.glob()` method.

Then, note that the hydrological data contains non-numeric values (missing samples), that have either been replaced in the file by the values -30000.0, NAN or 1e+10 (why would you like something simple?). We will have to deal with these missing values. For that, you can use the `na_values` keyword argument of the `pd.read_csv()` function. Please, check the documentation of this function to see how to use it. For convenience, you can also parse the dates automatically with the `parse_dates` keyword argument. In general, checking the documentation of `pd.read_csv()` is very instructive. You can find it [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).

> __Question 4.__ Load the hydrological data into a single DataFrame. Similarly to the chemical data, you should use the date and time information as the index of the DataFrame. What relevant information do you think you should extract from the hydrological data to perform the calibration? Plot one of them as a function of time.

In [ ]:
from glob import glob

# Your amazing code here

## 4. Prepare and inspect the data

### 4.1. Synchronize the data

You may have realized that the chemical and hydrological data are not synchronized. Actually, the chemical data is sparse in time, whereas the hydrological data is (more or less) continuous with a minimum of 1 measurement every 5 minutes. We will have to synchronize the data to be able to compare them.

> __Question 5.__ Resample the hydrological data to match the chemical data. You can use the `pd.DataFrame.resample()` method in order to do this. Take the time to read the documentation of this method, and to see implemented examples on the web.

In [ ]:
# Get the minimum common start and end dates
start = max(suspended_load.index.min(), data_hydro.index.min())
end = min(suspended_load.index.max(), data_hydro.index.max())

# Trucate the dataframes
suspended_load = suspended_load.truncate(before=start, after=end)
data_hydro_resampled = data_hydro.truncate(before=start, after=end).copy()

# Interpolate on a fine time grid and reindex on the most constraining index
data_hydro_resampled = data_hydro_resampled.resample("1min")
data_hydro_resampled = data_hydro_resampled.interpolate(method="linear", limit=10)

# Reindex
data_hydro_resampled = data_hydro_resampled.reindex(suspended_load.index)

# Gather the data in a single dataframe
data = data_hydro_resampled.copy()
data["suspended_load"] = suspended_load
data.dropna(inplace=True)

# Plot the data
ax = data.plot(
    subplots=True,
    style=".",
    grid=True,
    xlabel="Date",
    legend=False,
    ms=2,
);

# Labels
ax[0].set_ylabel("$h$ (cm)")
ax[1].set_ylabel("$T$ (NTU)")
ax[2].set_ylabel("$C_S$ (mg/L)");

### 4.2. Inspect the dataset

Now we can start to investigate correlations between the turbidity and the suspended load. We will start by plotting the data. We will also compute the correlation coefficient between the turbidity and the suspended load. The goal here is to get a first idea of the relationship between the turbidity and the suspended load. 

> __Question 6.__ Using the `pd.plotting.scatter_matrix()` function, inspect the relationship between the turbidity, water level and suspended load. What do you observe? Does the correlation coefficient give you a good idea of the relationship between the turbidity and the suspended load? 

## 5. Calibrate the turbidity probe with machine learning

In this section, we will try several machine learning models to find the best one to predict the suspended load from the turbidity. We will use the `scikit-learn` library. 

We first need to formalize the problem we are trying to solve. The goal here is to find the relationship between the turbidity $x$ and the suspended load $y$. In other terms, we want to find a function $f_\theta$ such that 

$$f_\theta(x) = y$$

This function is called a model, and its parameters are noted $\theta$ = $(\theta_1, \theta_2, \dots, \theta_n)$. We will try several models and compare them to find the best one, namely $\theta^*$ = $(\theta_1^*, \theta_2^*, \dots, \theta_n^*)$, that minimizes the error between the predicted suspended load and the measured suspended load.

### 5.1. First naive trial with linear regression

There is no real need to use the `scikit-learn` library to perform a linear regression. However, it is a good way to start to use the library and understand its _greybox_ approach. In the next cell, we will import a linear regression model, instantiate it, fit it to the data and plot the result. Following the notations above, the linear regression uses a model of the form

$$y = f_\theta(x) = \theta_1 x + \theta_0$$

The parameters $\theta_0$ and $\theta_1$ are called the intercept and the slope, respectively. The goal of the linear regression is to find the best values for these parameters. In other terms, we want to find the best line that fits the data.

> __Question 7.__ What is the error function minimized by the linear regression? What score do you obtain out of the linear regression? What does it mean? Do you think that splitting the data into a training and a testing set is useful here?

In [ ]:
# Rename vectors
y = np.ones(10)
x = np.zeros(10) # Turn turbidity into a numpy array here

# Reshape the vectors
y = y.reshape(-1, 1)
x = x.reshape(-1, 1)

# Create a linear regression object
model_1 = linear_model.LinearRegression()

# Fit the model
model_1.fit(x, y)

# Get predictions
y_pred = model_1.predict(x)

# Plot results

### 5.2. Robust linear regression

It is obvious that the linear regression struggles with outliers. Several solutions exist to deal with outliers. One of them is to use a robust linear regression, which performs the linear regression on a subset of the data and iteratively removes the outliers in a randomized way. The `scikit-learn` library provides a robust linear regression model. You can find more information about it in the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RANSACRegressor.html).

> __Question 8.__ Use the robust linear regression model to fit the data. What score do you obtain? How does it depend on the hyperparameters of the model? Do you think that splitting the data into a training and a testing set is useful here?

### 5.3. Working in the log-log space

As you were told quite a lot in the lecture, the representation of the data is very important. As you can understand from the scatter matrix plot that we did in the data preparation section, the density of measurements in higher for lower values of the suspended load. This is a problem for the linear regression, since it will tend to fit the data in the low range of suspended load. One way to deal with this problem is to work in the log-log space.

> __Question 9.__ Make the inspection of the data in the log-log space. What do you observe? What is the score of the linear regression in the log-log space? What is the score of the robust linear regression in the log-log space? 

### 5.4. Using a multivariate linear regression

We have seen that the water level is also correlated with the suspended load. It is therefore interesting to use it as an additional feature to predict the suspended load. This is called a multivariate linear regression. The `scikit-learn` library provides a multivariate linear regression model. You can find more information about it in the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html).

> __Question 10.__ Use the multivariate linear regression model to fit the data. What score do you obtain? How does it depend on the hyperparameters of the model? 

### 5.5. Testing other models

Now we can use the `scikit-learn` library to test other models. You can find a list of the available models in the [documentation](https://scikit-learn.org/stable/supervised_learning.html). You can also find a list of the available metrics in the [documentation](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics). This is a good opportunity to understand the concept of greybox. You can also use the `scikit-learn` [cheat sheet](https://scikit-learn.org/stable/machine_learning_map.html) to help you choose the right model.

![](https://scikit-learn.org/stable/_static/ml_map.png)

> __Question 11.__ Test other models and compare them. What is the best model, and why? Consider using a train-test split to answer this question. Argue your answer.

## 6. Use your model to predict the suspended load on a time period where no suspended load measurements are available

Now it's time to use your model to predict the suspended load on a time period where no suspended load measurements are available. Based on the model comparison you did in the previous section, choose the best model and train it on the whole dataset. Then, use it to predict the suspended load on the period from 2015-03-01 to 2015-05-01. 

> __Question 12.__ Plot the predicted suspended load and the measured suspended load on the same plot. What do you observe? What special patterns do observe in the predicted suspended load that you do not observe in the measured suspended load? Try to explain these patterns in your own words.